In [1]:
from selenium import webdriver
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import bs4, time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
pd.options.display.max_rows = 1000
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None

In [2]:
clubes = pd.read_csv('clubes_data.csv', low_memory=False)
mundial = pd.read_csv('mundial_data.csv', low_memory=False)

In [26]:
def getGames(market, data_hoje, games_now=False, intervalo=False):
    '''
        Ira pegar os ultimos jogos
        
        games_now para pegar somente os jogos de agora
    '''

    total = 0
    count = 0
    
    if market == 'clubes':
        main_url = 'https://www.betfair.com/sport/virtuals-results?sport=SOCCER&day='
    elif market == 'mundial':
        main_url = 'https://www.betfair.com/sport/virtuals-results?sport=SOCCER_WORLD_CUP&day='

    link =  main_url + data_hoje
        
    if games_now != False:
        req = requests.get(link)
        if not req.status_code == 200:
            return 'Problem with request'
        else:
            htmlPage = bs4.BeautifulSoup(req.content, features='html.parser')
    else:
        options = Options()
#         options.add_argument('--headless')
        options.add_argument("user-data-dir=./bin/profile_browser")
        
        if not("driver" in globals()):
            global driver
            driver = webdriver.Chrome(executable_path='./bin/chromedriver.exe', chrome_options=options)
        
        driver.get(link)
        time.sleep(1)
        while total != '475':
            total = driver.find_element_by_class_name('number-of-results').text
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            time.sleep(1)
            driver.find_element_by_tag_name('html').send_keys(Keys.HOME)
            count +=1
            if count >=13 and intervalo==False:
#                 break
                print('\ndia com menos de 475\n')
                next_or_break = input('1 para parar de tentar pegar os dados ou outra tecla para continuar\n:')
                if str(next_or_break) == '1':
                    driver.close()
                    break  
            elif count >=13 and intervalo==True:
                break
        htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
        
    
    table_results = htmlPage.find('div',{'class':'result-list-wrapper'})

    data_day = []
    for event in table_results.find_all('div',{'class':'result-title'}):
        content = event.text.split('\n')
        event_data = {'date': '', 'time': '', 'team_home': '', 'score': '', 'team_back': ''}

        event_data['date'] = data_hoje
        event_data['time'] = content[1]
        event_data['team_home'] = content[2]
        event_data['score'] = content[4]
        event_data['team_back'] = content[6]

        data_day.append(event_data)

    data = pd.DataFrame(data_day, columns=list(event_data.keys()))
    print('saved day: {}'.format(data_hoje))

    return data

In [53]:
a = getGames('clubes', '2020-07-04', intervalo=True)
a = cleanDatetime(a)

saved day: 2020-07-04


In [4]:
def cleanDatetime(data):
    '''
        Pre processa os dados e os deixa prontos para analise
    '''
    for row in range(len(data)):
        dateAndTime = pd.to_datetime(data.loc[row, 'date'] + " " + data.loc[row, 'time'])
    
        if dateAndTime.hour > 18:
            dateAndTime = dateAndTime - timedelta(days=1)
        
        dateAndTime = dateAndTime + timedelta(hours=2)
        data.loc[row, 'datetime'] = dateAndTime
        
        if row % 1000 == 0: print(row)
        
    data.drop(['date', 'time'], axis=1, inplace=True)
    data.sort_values(['datetime'], inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

In [5]:
def getNewData(data, market):
    '''
        Recebe o dataframe antigo
        Adiciona os dados mais recentes ao dataframe
    '''
    last_item = len(data)-1
    data.datetime = pd.to_datetime(data['datetime'])
    difference = (datetime.today() - data.loc[last_item, 'datetime']).days
    
    new_games = pd.DataFrame()
    saved_days = []
    for i in range(difference+1):
        day = ((datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d'))
        if day not in saved_days:
            new_games = new_games.append(getGames(market, day, intervalo=True))
            
    new_games.reset_index(drop=True, inplace=True)
    new_games = cleanDatetime(new_games)
    return new_games

In [6]:
def concatDF(old_df, new_df):
    new = pd.concat([old_df, new_df], axis=0, sort=True)
    new.sort_values(['datetime'], inplace=True)
    new.drop_duplicates(inplace=True)
    new.reset_index(drop=True, inplace=True)
    return new

In [19]:
new_mundial = getNewData(mundial, 'mundial')
new_clubes = getNewData(clubes, 'clubes')

In [58]:
clubes.head()

,datetime,score,team_back,team_home
0,2020-02-11 21:02:00,1 - 1,Torino Bulls,Sassuolo Villa
1,2020-02-11 21:05:00,0 - 1,Turin,Munich
2,2020-02-11 21:08:00,2 - 2,Biancorossi,Udine Utd
3,2020-02-11 21:11:00,1 - 0,Swansea Jacks,London Whites
4,2020-02-11 21:14:00,1 - 0,Islington Reds,East End Utd


In [59]:
clubes.tail()

,datetime,score,team_back,team_home
68926,2020-07-06 10:20:00,2 - 0,Chelsea Pensioners,Islington Reds
68927,2020-07-06 10:23:00,1 - 2,Liverpool Stars,East End Utd
68928,2020-07-06 10:26:00,2 - 0,Chelsea Pensioners,London Whites
68929,2020-07-06 10:29:00,3 - 0,Verona City,Milan Reds
68930,2020-07-06 10:32:00,2 - 2,Turin,Verona City


In [96]:
# mundial.to_csv('./mundial_data.csv', index=None)
# clubes.to_csv('./clubes_data.csv', index=None)

In [93]:
# mundial = concatDF(mundial, new_mundial)
# clubes = concatDF(new, new_clubes)

In [101]:
len(clubes)

95599

In [98]:
len(mundial)

101283

In [102]:
mundial.tail()

,datetime,score,team_back,team_home
101278,2020-08-31 10:22:00,1 - 1,England,Portugal
101279,2020-08-31 10:25:00,1 - 1,Japan,Iran
101280,2020-08-31 10:28:00,4 - 0,Russia,Portugal
101281,2020-08-31 10:31:00,3 - 1,Chile,England
101282,2020-08-31 10:34:00,3 - 1,Italy,England
